# Tasking Data Processing
To pull from Splunk:

```
index="" host="" sourcetype=""
| search "Received Status Update"
| rex field=_raw "itemIds:\[(?<skuNo>\d+)\]"
| rex field=_raw "pickLocationEmptyConfidence:.*?statusCode:(?<statusCode>\d+), statusDetail:,"
| rex field=_raw "customerTaskId:(?<taskID>\w+), itemIds"
| rex field=_raw "customerTaskId:\d{3}_(?<stationID>\d+)_"
| rex field=_raw "\"suction_cup_name\":\"(?<suctionCupName>\w+)\""
| rex field=_raw "commandUtime:(?<ctime>\d+)\}"
| eval commandTime = strftime(ctime/1000000,"%Y-%m-%d %H:%M:%S")
| dedup skuNo, statusCode, taskID, ctime keepempty=true consecutive=false
| table skuNo, statusCode, stationID, taskID, suctionCupName, ctime, commandTime
```

# Process the raw tasking data
* Read the data from csv
* Get the cup configuration from the previous task
* Remove UPC error and pause state

In [22]:
import csv
import os
import pathlib
import datetime
fname = 'rhr_tasks_0201_0911.csv'
fpath = pathlib.Path(os.getcwd()) / fname

In [23]:
class Task:
    def __init__(self, sku, status, stationId, taskId, suctionCup, ctime, commandTime):
        self.sku = sku
        self.status = status
        self.stationId = stationId
        self.taskId = taskId
        self.suctionCup = suctionCup
        self.ctime = int(ctime)
        self.commandTime = datetime.datetime.strptime(commandTime, '%Y-%m-%d %H:%M:%S')
        
    def __str__(self):
        return ','.join([k+':'+str(v) for k, v in vars(self).items()])

In [24]:
raws = []
with open(fpath, 'r') as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        if row[0] == '' or row[0] is None:
            continue
        try:
            raws.append(Task(row[0], row[1], row[2], row[3], row[4], row[5], row[6]))
        except Exception as e:
            print(e)

invalid literal for int() with base 10: 'ctime'


In [25]:
# sort raw
def sort_raw(raw:Task):
    return (raw.stationId, raw.ctime)
raws = sorted(raws, key=sort_raw)

In [26]:
len(raws)

276465

In [27]:
[print(x) for x in raws[0:5]]

sku:2145184,status:200399,stationId:44,taskId:173_44_14638577_19570783_1_13713437_33151_1478039_1_402546_1_1,suctionCup:,ctime:1686581226633130,commandTime:2023-06-12 10:47:00
sku:2145184,status:200302,stationId:44,taskId:173_44_14638581_19570798_1_13713437_33151_1478039_1_402547_1_1,suctionCup:small,ctime:1686581351901894,commandTime:2023-06-12 10:49:00
sku:2145184,status:200399,stationId:44,taskId:173_44_14638581_19570798_1_13713437_33151_1478039_1_402547_1_1,suctionCup:,ctime:1686581356220168,commandTime:2023-06-12 10:49:00
sku:24402489,status:200302,stationId:44,taskId:173_44_14638487_19569868_1_13713433_319_1471834_1_402606_1_1,suctionCup:mini,ctime:1686581726849315,commandTime:2023-06-12 10:55:00
sku:24402489,status:200399,stationId:44,taskId:173_44_14638487_19569868_1_13713433_319_1471834_1_402606_1_1,suctionCup:,ctime:1686581738293651,commandTime:2023-06-12 10:55:00


[None, None, None, None, None]

In [28]:
# remove pause and UPC error tasks and take the suction cup from previous tasks
tasks = []
suctionCup = None
for task in raws:
    suctionCup = task.suctionCup if task.suctionCup != '' else suctionCup
    if task.status != '200302' and task.status != '200451':
        tasks.append(Task(task.sku, task.status, task.stationId, task.taskId, suctionCup, task.ctime, datetime.datetime.strftime(task.commandTime, '%Y-%m-%d %H:%M:%S')))        

In [29]:
len(tasks)

142456

In [30]:
[print(x) for x in tasks[0:10]]

sku:2145184,status:200399,stationId:44,taskId:173_44_14638577_19570783_1_13713437_33151_1478039_1_402546_1_1,suctionCup:None,ctime:1686581226633130,commandTime:2023-06-12 10:47:00
sku:2145184,status:200399,stationId:44,taskId:173_44_14638581_19570798_1_13713437_33151_1478039_1_402547_1_1,suctionCup:small,ctime:1686581356220168,commandTime:2023-06-12 10:49:00
sku:24402489,status:200399,stationId:44,taskId:173_44_14638487_19569868_1_13713433_319_1471834_1_402606_1_1,suctionCup:mini,ctime:1686581738293651,commandTime:2023-06-12 10:55:00
sku:24408522,status:200399,stationId:44,taskId:173_44_14638572_19570752_1_13713440_10574_1479464_1_402567_1_1,suctionCup:large_type2,ctime:1686582213989421,commandTime:2023-06-12 11:03:00
sku:24408522,status:200399,stationId:44,taskId:173_44_14638583_19570806_1_13713440_10574_1479464_1_402568_1_1,suctionCup:large_type2,ctime:1686582248988658,commandTime:2023-06-12 11:04:00
sku:24408522,status:200441,stationId:44,taskId:173_44_14638586_19570890_1_13713440_1

[None, None, None, None, None, None, None, None, None, None]

# Aggregate the data into SKU/CUP level for the AB test

First need to calculate by SKU and cup, what is the success rate 

In [31]:
import pandas as pd

In [32]:
df_tasks = pd.DataFrame([vars(t) for t in tasks])

In [33]:
df_tasks.head()

,sku,status,stationId,taskId,suctionCup,ctime,commandTime
0,2145184,200399,44,173_44_14638577_19570783_1_13713437_33151_1478...,None,1686581226633130,2023-06-12 10:47:00
1,2145184,200399,44,173_44_14638581_19570798_1_13713437_33151_1478...,small,1686581356220168,2023-06-12 10:49:00
2,24402489,200399,44,173_44_14638487_19569868_1_13713433_319_147183...,mini,1686581738293651,2023-06-12 10:55:00
3,24408522,200399,44,173_44_14638572_19570752_1_13713440_10574_1479...,large_type2,1686582213989421,2023-06-12 11:03:00
4,24408522,200399,44,173_44_14638583_19570806_1_13713440_10574_1479...,large_type2,1686582248988658,2023-06-12 11:04:00


In [34]:
df_tasks['success'] = df_tasks['status'].apply(lambda x: True if x == '200399' else False)

In [35]:
df_tasks.sample(10)

,sku,status,stationId,taskId,suctionCup,ctime,commandTime,success
52738,710210,200399,44,173_44_16037580_21327224_1_15080448_28662_1581...,large_type2,1692313169558611,2023-08-17 18:59:00,True
38975,209882,200441,44,173_44_15726133_20961680_1_14785454_20274_1597...,large_type2,1691154979740740,2023-08-04 09:16:00,False
81854,559215,200399,45,174_45_14836841_19857162_1_13910400_34056_1499...,small,1687456758720408,2023-06-22 13:59:00,True
101185,513186,200441,45,174_45_15431404_20608387_1_14494338_16336_1538...,mini,1690213400317618,2023-07-24 11:43:00,False
24732,24388084,200399,44,173_44_15365920_20543533_1_14421118_17565_1558...,large_type2,1689862230777579,2023-07-20 10:10:00,True
32020,24384008,200399,44,173_44_15569548_20778863_1_14627340_26097_1459...,small,1690564173418246,2023-07-28 13:09:00,True
121293,166751,200399,45,174_45_16021228_21312363_1_15068872_13264_1620...,large_type2,1692289246694315,2023-08-17 12:20:00,True
125820,854518,200450,45,174_45_16183767_21491817_1_15219666_36281_1628...,small,1692876338891881,2023-08-24 07:25:00,False
5797,2140574,200399,44,173_44_14829211_19848735_1_13904024_32795_1493...,small,1687436515312187,2023-06-22 08:21:00,True
26576,917881,200399,44,173_44_15418053_20600716_1_14472209_20857_1555...,large_type2,1689978919089820,2023-07-21 18:35:00,True


In [36]:
df = df_tasks.groupby(['sku','suctionCup'], sort=True).agg(p=pd.NamedAgg(column='success', aggfunc=lambda x: sum(x)/len(x)), n=pd.NamedAgg(column='success', aggfunc='count'))
df = df.reset_index()
df = df.sort_values(by='n', ascending=False)

In [37]:
df.head(20)

,sku,suctionCup,p,n
1154,2030289,large_type2,0.644594,1933
2109,24388084,large_type2,0.901363,1541
5643,512663,small,0.997188,1067
2116,24388284,small,0.875648,965
1360,2145184,small,0.933893,953
8070,917881,large_type2,0.923341,874
610,1611435,small,0.638539,794
1264,209882,large_type1,0.745718,759
1358,2145184,large_type2,0.968839,706
2115,24388284,large_type2,0.948760,605


## Need to find the SKUs picked with more than one cup 

In [38]:
df_filtered = df[(df['p'] > 0) & (df['n'] >= 20)]

In [39]:
df_filtered = df_filtered[df_filtered['sku'].duplicated(keep=False)].sort_values(by='sku', ascending=False)

In [40]:
df_filtered = df_filtered[df_filtered['sku'].duplicated(keep=False)].sort_values(by='n', ascending=False)

In [41]:
df_filtered[0:30]

,sku,suctionCup,p,n
5643,512663,small,0.997188,1067
2116,24388284,small,0.875648,965
1360,2145184,small,0.933893,953
610,1611435,small,0.638539,794
1264,209882,large_type1,0.745718,759
1358,2145184,large_type2,0.968839,706
2115,24388284,large_type2,0.948760,605
2615,24430099,small,0.761194,536
2609,24430096,small,0.878505,535
8050,917860,large_type2,0.802817,426


### SKU 24430099
* baseline = 0.855932
* dmin = baseline*0.15
* alpha = 0.05
* power = 0.8

Size = 121

In [42]:
df_filtered[df_filtered['sku'] == '24430099']

,sku,suctionCup,p,n
2615,24430099,small,0.761194,536
2613,24430099,large_type2,0.554545,220
2614,24430099,mini,0.681818,44


### SKU 1611435
* baseline = 0.787136
* dmin = baseline*0.15
* alpha = 0.05
* power = 0.8

Size = 196

In [43]:
df_filtered[df_filtered['sku'] == '1611435']

,sku,suctionCup,p,n
610,1611435,small,0.638539,794
608,1611435,large_type2,0.555985,259
609,1611435,mini,0.670455,88


In [44]:
sku_counts = df_filtered['sku'].value_counts()

# Filter SKUs that have appeared more than once
skus_appeared_more_than_once = sku_counts[sku_counts > 1]
len(skus_appeared_more_than_once)

# only 201 skus has two suction cup configured and has at least 20 sample points per group

201

## Find SKUs that have enough sample points to be compared in AB test.

In [45]:
from ABTestToolKit import get_sample_size


alpha = 0.05
power = 0.8
eligible_combo = []


for sku in df_filtered['sku'].unique():
    df_sku = df_filtered[df_filtered['sku'] == sku]
    n_cup = len(df_sku)
    
    for i in range(n_cup-1):
        con = df_sku[df_sku['p'] == df_sku['p'].max()]
        exp = df_sku[df_sku['p'] == df_sku['p'].nlargest(i+2).values[-1]]

        # param
        n_ctl = con['n'].values[0]
        n_exp = exp['n'].values[0]
        baseline = con['p'].values[0]
        p_exp = exp['p'].values[0]
        dmin = baseline*0.15

        # calculate size for AB test
        try:
            n = get_sample_size(alpha, power, dmin, baseline)
        except Exception as e:
            print(f"sku {sku} couldn't be measured with bl {baseline} and dmin {dmin}")
            continue

        # add eligible skus to the list
        if con['n'].values[0] >= n and exp['n'].values[0] >= n:
            print(f"sku {sku} can be measured with bl {baseline}, dmin {dmin} and n {n} ")
            eligible_combo.append((sku, n_ctl, n_exp, baseline, p_exp, dmin))    

sku 512663 can be measured with bl 1.0, dmin 0.15 and n 38 
sku 24388284 can be measured with bl 0.9487603305785124, dmin 0.14231404958677685 and n 65 
sku 2145184 can be measured with bl 0.9688385269121813, dmin 0.1453257790368272 and n 54 
sku 209882 can be measured with bl 0.7457180500658761, dmin 0.11185770750988142 and n 211 
sku 24430099 can be measured with bl 0.7611940298507462, dmin 0.11417910447761193 and n 197 
sku 24430096 can be measured with bl 0.9690721649484536, dmin 0.14536082474226802 and n 54 
sku 737194 can be measured with bl 0.9858156028368794, dmin 0.1478723404255319 and n 45 
sku 2498462 can be measured with bl 0.9701897018970189, dmin 0.14552845528455283 and n 53 
sku 24460346 can be measured with bl 0.9465408805031447, dmin 0.1419811320754717 and n 66 
sku 41302 can be measured with bl 0.883177570093458, dmin 0.1324766355140187 and n 105 
sku 487129 can be measured with bl 0.828125, dmin 0.12421874999999999 and n 143 
sku 633858 can be measured with bl 0.96296

In [46]:
len(eligible_combo)

28

In [47]:
print("sku, n_con, n_exp, p_con, p_exp")
for x in eligible_combo:
    print(x[0], x[1], x[2], x[3], x[4])

sku, n_con, n_exp, p_con, p_exp
512663 300 1067 1.0 0.9971883786316776
24388284 605 965 0.9487603305785124 0.8756476683937824
2145184 706 953 0.9688385269121813 0.9338929695697796
209882 759 295 0.7457180500658761 0.6271186440677966
24430099 536 220 0.7611940298507462 0.5545454545454546
24430096 97 535 0.9690721649484536 0.8785046728971962
737194 141 375 0.9858156028368794 0.9733333333333334
2498462 369 74 0.9701897018970189 0.9054054054054054
24460346 318 81 0.9465408805031447 0.9382716049382716
41302 214 134 0.883177570093458 0.29850746268656714
487129 192 194 0.828125 0.8195876288659794
633858 189 74 0.9629629629629629 0.8378378378378378
24395762 187 174 0.9411764705882353 0.7758620689655172
918408 75 181 0.9866666666666667 0.9668508287292817
483018 123 170 0.9512195121951219 0.9176470588235294
24414102 132 78 0.9621212121212122 0.9615384615384616
24414102 132 72 0.9621212121212122 0.9444444444444444
504688 128 100 0.953125 0.95
220061 104 126 0.9423076923076923 0.7142857142857143
2

In [48]:
# show all eligible skus:
df_filtered[df_filtered['sku'].isin([x[0] for x in eligible_combo])]

,sku,suctionCup,p,n
5643,512663,small,0.997188,1067
2116,24388284,small,0.875648,965
1360,2145184,small,0.933893,953
1264,209882,large_type1,0.745718,759
1358,2145184,large_type2,0.968839,706
2115,24388284,large_type2,0.948760,605
2615,24430099,small,0.761194,536
2609,24430096,small,0.878505,535
6981,737194,large_type1,0.973333,375
3405,2498462,large_type2,0.970190,369


## Perform AB test on the SKUs with enough data points.


* Take the suction cup with higher success rate as the control group
* Take the suction cup with lower success rate as the experiment group
* Perform AB test to confirm if they have 15% relative difference
* If the result is significant, check how many failed attempts can be converted into success.

In [49]:
from ABTestToolKit import get_two_sample_Z_test

significants = []

for combo in eligible_combo:
    sku = combo[0]
    
    # param
    baseline = combo[3]
    dmin = combo[5]
    n_ctl = combo[1]
    x_ctl =  n_ctl * baseline
    n_exp = combo[2]
    x_exp = n_exp * combo[4]
    alpha = 0.05
    
    
    # ab test
    _, p, _, _, _ = get_two_sample_Z_test(n_ctl, x_ctl, n_exp, x_exp, alpha)
    
    if p < alpha:
        n_convert = int(n_exp * baseline - x_exp)
        significants.append((sku, n_exp, n_convert))
        print(f"significant sku {sku}, can convert {n_convert} from failure to success")

control mean: 1.0
experiment mean: 0.9971883786316776
The confidence interval for the difference is [-0.0022, 0.0028, 0.0078]
statistics: 0.919424594997516
The p value: 0.3578735297045885
control mean: 0.9487603305785124
experiment mean: 0.8756476683937824
The confidence interval for the difference is [0.0480, 0.0731, 0.0983]
statistics: 4.781939713261551
The p value: 1.7361175121966e-06
significant sku 24388284, can convert 70 from failure to success
control mean: 0.9688385269121813
experiment mean: 0.9338929695697796
The confidence interval for the difference is [0.0169, 0.0349, 0.0530]
statistics: 3.1919210318507854
The p value: 0.0014132996609517168
significant sku 2145184, can convert 33 from failure to success
control mean: 0.7457180500658761
experiment mean: 0.6271186440677966
The confidence interval for the difference is [0.0675, 0.1186, 0.1697]
statistics: 3.819384950384808
The p value: 0.00013378484036374871
significant sku 209882, can convert 34 from failure to success
contr

In [50]:
significants

[('24388284', 965, 70),
 ('2145184', 953, 33),
 ('209882', 295, 34),
 ('24430099', 220, 45),
 ('24430096', 535, 48),
 ('2498462', 74, 4),
 ('41302', 134, 78),
 ('633858', 74, 9),
 ('24395762', 174, 28),
 ('220061', 126, 28),
 ('123729', 84, 8),
 ('867590', 45, 4),
 ('24402489', 69, 7),
 ('807969', 56, 9),
 ('2126870', 48, 10)]

In [51]:
len(significants)

15